# ZVP-based reverse-engineering

In [ ]:
import sympy
from sympy import FF, sympify, symbols, Poly, Monomial
from collections import Counter
import tabulate
from IPython.display import HTML, display
from tqdm.notebook import tqdm

from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters
from pyecsca.ec.formula import FormulaAction
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod
from pyecsca.ec.context import DefaultContext, local
from pyecsca.ec.mult import LTRMultiplier, AccumulationOrder
from pyecsca.misc.cfg import getconfig
from pyecsca.sca.re.zvp import unroll_formula, compute_factor_set, zvp_points

In [ ]:
cfg = getconfig()
cfg.ec.mod_implementation = "python"

In [ ]:
model = ShortWeierstrassModel()
coordsaff = AffineCoordinateModel(model)
coords = model.coordinates["projective"]
add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
neg = coords.formulas["neg"]

# A 64-bit prime order curve for testing things out
p = 0xc50de883f0e7b167
a = Mod(0x4833d7aa73fa6694, p)
b = Mod(0xa6c44a61c5323f6a, p)
gx = Mod(0x5fd1f7d38d4f2333, p)
gy = Mod(0x21f43957d7e20ceb, p)
n = 0xc50de885003b80eb
h = 1

field = FF(p)

infty = Point(coords, X=Mod(0, p), Y=Mod(1, p), Z=Mod(0, p))
g = Point(coords, X=gx, Y=gy, Z=Mod(1, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a,b=b))
params = DomainParameters(curve, g, n, h)

In [ ]:
adds = list(filter(lambda formula: formula.name.startswith("add"), coords.formulas.values()))

In [ ]:
gens = None
factor_sets = {}
for add in adds:
    factor_set = compute_factor_set(add, affine=True)
    factor_sets[add.name] = factor_set

In [ ]:
c = Counter()
for factor_set in factor_sets.values():
    for poly in factor_set:
        c[poly] += 1
print(c)

In [ ]:
def simulated_oracle(scalar, affine_point):
    real_coords = model.coordinates["projective"]
    real_add = real_coords.formulas["add-2007-bl"]
    real_dbl = real_coords.formulas["dbl-2007-bl"]
    real_mult = LTRMultiplier(real_add, real_dbl, None, False, AccumulationOrder.PeqPR, True, True)
    point = affine_point.to_model(params.curve.coordinate_model, params.curve)
    with local(DefaultContext()) as ctx:
        real_mult.init(params, point)
        real_mult.multiply(scalar)

    trace = []

    def callback(action):
        if isinstance(action, FormulaAction):
            for intermediate in action.op_results:
                trace.append(intermediate.value)
    ctx.actions.walk(callback)
    return any(int(value) == 0 for value in trace)

In [ ]:
table = [["Formula", "Polynomial", "k", "Oracle output"]]

big_scalar = 7  # -> dbl(1) = 2, add(1, 2) = 3, dbl(3) = 6, add(1, 6) = 7
small_scalars = [2,6]
for formula_name, factor_set in tqdm(factor_sets.items()):
    for poly in tqdm(factor_set):
        for small_scalar in small_scalars:
            pts = zvp_points(poly, curve, small_scalar, n)
            if pts:
                outputs = []
                for point in pts:
                    #print(formula_name, poly, small_scalar, simulated_oracle(big_scalar, point))
                    outputs.append(simulated_oracle(big_scalar, point))
                table.append([formula_name, poly, small_scalar, outputs])
display(HTML(tabulate.tabulate(table, tablefmt="html", headers="firstrow")))